In [1]:
import os
import requests
from getpass import getpass
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [2]:
OPENAI_API_KEY = getpass("Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [3]:
class PDFDownloader:
    def __init__(self, urls):
        self.urls = urls

    def download(self):
        ml_papers = []
        for i, url in enumerate(self.urls):
            response = requests.get(url)
            if response.status_code == 200:
                filename = f"paper{i+1}.pdf"
                with open(filename, 'wb') as f:
                    f.write(response.content)
                    print(f"Downloaded {filename}")
                ml_papers.append(filename)
            else:
                print(f"Failed to download {url}")
        return ml_papers

In [5]:
# Uso de la clase PDFDownloader

urls = [
    'https://arxiv.org/pdf/2306.06031v1.pdf',  # Corrected URL
    'https://arxiv.org/pdf/2306.12156v1.pdf',
    'https://arxiv.org/pdf/2306.14289v1.pdf',
    'https://arxiv.org/pdf/2305.10973v1.pdf',
    'https://arxiv.org/pdf/2306.13643v1.pdf',
]

downloader = PDFDownloader(urls)
ml_papers = downloader.download()

print("Contenido de ml_papers:")
print(ml_papers)

Downloaded paper1.pdf
Downloaded paper2.pdf
Downloaded paper3.pdf
Downloaded paper4.pdf
Downloaded paper5.pdf
Contenido de ml_papers:
['paper1.pdf', 'paper2.pdf', 'paper3.pdf', 'paper4.pdf', 'paper5.pdf']


In [6]:
class PDFLoader:
    def __init__(self, filenames):
        self.filenames = filenames

    def load(self):
        documents = []
        for filename in self.filenames:
            loader = PyPDFLoader(filename)
            documents.extend(loader.load())
        return documents

In [7]:
# Uso de la clase PDFLoader

loader = PDFLoader(ml_papers)
documents = loader.load()

print("Contenido de documents:")
print(documents)

Contenido de documents:
[Document(page_content='FinGPT: Open-Source Financial Large Language Models\nHongyang (Bruce) Yang1, Xiao-Yang Liu1, Christina Dan Wang2\n1Columbia University;2New York University (Shanghai)\n{HY2500, XL2427 }@columbia.edu; christina.wang@nyu.edu\nAbstract\nLarge language models (LLMs) have shown the\npotential of revolutionizing natural language pro-\ncessing tasks in diverse domains, sparking great\ninterest in finance. Accessing high-quality fi-\nnancial data is the first challenge for financial\nLLMs (FinLLMs). While proprietary models\nlike BloombergGPT have taken advantage of their\nunique data accumulation, such privileged access\ncalls for an open-source alternative to democratize\nInternet-scale financial data.\nIn this paper, we present an open-source large\nlanguage model, FinGPT, for the finance sec-\ntor. Unlike proprietary models, FinGPT takes a\ndata-centric approach, providing researchers and\npractitioners with accessible and transparent re-\nso

In [8]:
class DocumentSplitter:
    def __init__(self, documents):
        self.documents = documents

    def split(self):
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1500, chunk_overlap=200, length_function=len)
        return text_splitter.split_documents(self.documents)

In [9]:
# Uso de la clase DocumentSplitter

splitter = DocumentSplitter(documents)
documents = splitter.split()

print("Contenido de documents:")
print(documents)

Contenido de documents:
[Document(page_content='FinGPT: Open-Source Financial Large Language Models\nHongyang (Bruce) Yang1, Xiao-Yang Liu1, Christina Dan Wang2\n1Columbia University;2New York University (Shanghai)\n{HY2500, XL2427 }@columbia.edu; christina.wang@nyu.edu\nAbstract\nLarge language models (LLMs) have shown the\npotential of revolutionizing natural language pro-\ncessing tasks in diverse domains, sparking great\ninterest in finance. Accessing high-quality fi-\nnancial data is the first challenge for financial\nLLMs (FinLLMs). While proprietary models\nlike BloombergGPT have taken advantage of their\nunique data accumulation, such privileged access\ncalls for an open-source alternative to democratize\nInternet-scale financial data.\nIn this paper, we present an open-source large\nlanguage model, FinGPT, for the finance sec-\ntor. Unlike proprietary models, FinGPT takes a\ndata-centric approach, providing researchers and\npractitioners with accessible and transparent re-\nso

In [10]:
# Esta clase crea embeddings (representaciones vectoriales) de los segmentos de texto utilizando OpenAIEmbeddings. Luego, almacena estos embeddings en una base de datos vectorial (Chroma) y prepara un objeto retriever para la búsqueda de información.

class DocumentEmbedder:
    def __init__(self, documents):
        self.documents = documents
        self.model = "text-embedding-ada-002"
        self.embeddings = OpenAIEmbeddings(model=self.model)

    def embed(self):
        vectorstore = Chroma.from_documents(
            documents=self.documents, embedding=self.embeddings)
        return vectorstore.as_retriever(search_kwargs={"k": 3})

In [11]:
# Uso de la clase DocumentEmbedder

embedder = DocumentEmbedder(documents)
retriever = embedder.embed()

print("Contenido de retriever:")
print(retriever)

/Users/adrianinfantes/Library/Caches/pypoetry/virtualenvs/pdfsummerizerlangchain-bBVf8GCi-py3.11/lib/python3.11/site-packages/chromadb/utils/embedding_functions.py:613: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self._task_type is "RETRIEVAL_DOCUMENT":


Contenido de retriever:
tags=['Chroma', 'OpenAIEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x12ff78950> search_kwargs={'k': 3}


In [12]:
# Esta clase establece y maneja el proceso de consulta y respuesta. Utiliza el modelo de chat de OpenAI junto con el objeto retriever para responder preguntas basadas en la información contenida en los documentos procesados.

class QAChain:
    def __init__(self, retriever):
        self.retriever = retriever
        self.openai_api_key = os.environ["OPENAI_API_KEY"]
        self.model_name = "gpt-3.5-turbo"
        self.temperature = 0.0
        self.chat = ChatOpenAI(
            openai_api_key=self.openai_api_key,
            model_name=self.model_name,
            temperature=self.temperature)
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.chat, chain_type="stuff", retriever=self.retriever)

    def query(self, question):
        return self.qa_chain.run(question)

In [13]:
# Uso de la clase QAChain

chain = QAChain(retriever)
answer = chain.query("qué es fingpt?")
print(answer)

FinGPT es un modelo de lenguaje de inteligencia artificial diseñado específicamente para aplicaciones financieras. Es un modelo de lenguaje de gran escala que utiliza técnicas de aprendizaje automático para comprender y generar texto relacionado con el ámbito financiero. FinGPT se centra en la curación y procesamiento de datos financieros de alta calidad y ofrece una amplia cobertura del mercado financiero. Además, FinGPT es un proyecto de código abierto y tiene como objetivo fomentar la innovación y desbloquear nuevas oportunidades en el campo de las finanzas abiertas.


PDFDownloader: Esta clase se encarga de descargar archivos PDF de una lista de URLs y guardarlos localmente. La función download realiza esta tarea y devuelve una lista de nombres de archivos descargados.

PDFLoader: Esta clase carga los documentos PDF guardados en memoria. Utiliza PyPDFLoader de LangChain para leer los archivos PDF y los almacena en una lista de documentos.

DocumentSplitter: La clase DocumentSplitter toma los documentos cargados y los divide en segmentos más manejables utilizando RecursiveCharacterTextSplitter. Esto es útil para el procesamiento posterior, especialmente para documentos grandes.

DocumentEmbedder: Esta clase crea embeddings (representaciones vectoriales) de los segmentos de texto utilizando OpenAIEmbeddings. Luego, almacena estos embeddings en una base de datos vectorial (Chroma) y prepara un objeto retriever para la búsqueda de información.

QAChain: Finalmente, la clase QAChain establece y maneja el proceso de consulta y respuesta. Utiliza el modelo de chat de OpenAI junto con el objeto retriever para responder preguntas basadas en la información contenida en los documentos procesados.